# System Pipeline: Load Preprocessed Data, Importance Scores, and Redundancy

This notebook demonstrates the complete pipeline for loading:
1. Preprocessed data for Building 1
2. Importance metrics from saved scores
3. Redundancy data

This is a streamlined version that loads pre-computed data without re-running expensive computations.

## Setup: Import Required Libraries

In [1]:
import sys
import subprocess

print(f"Current Python: {sys.executable}")

# Install packages in the current notebook environment
subprocess.check_call([sys.executable, "-m", "pip", "install", "openjij", "dwave-ocean-sdk", "-q"])

print("\n✓ Packages installed!")
print("Now restart your kernel: Kernel → Restart Kernel")
print("Then re-run all cells")

Current Python: C:\Users\Mohamed Khalil\Desktop\Quantum-Optimization-In-AP-Selection\venv\Scripts\python.exe

✓ Packages installed!
Now restart your kernel: Kernel → Restart Kernel
Then re-run all cells


In [2]:
# Add project root to Python path
import sys
from pathlib import Path

# Get the project root (2 levels up from this notebook)
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

print(f"✓ Added project root to Python path: {project_root}")

✓ Added project root to Python path: C:\Users\Mohamed Khalil\Desktop\Quantum-Optimization-In-AP-Selection


In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
import warnings

# Import custom data loading functions
from scripts.data.data_loaders import (
    load_preprocessed_data,
    load_all_precomputed_data,
    load_importance_dict_from_csv,
    load_redundancy_matrix_from_csv
)

# Import QUBO optimization functions
from scripts.optimization.QUBO import (
    formulate_qubo,
    solve_qubo_with_openjij,
    solve_qubo_with_SA
)

# Import ML training functions
from scripts.ml.ML_post_processing import train_regressor

# Import evaluation functions
from scripts.evaluation.Analysis import calculate_comprehensive_metrics

warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Step 1: Load Preprocessed Data for Building 1

This loads the preprocessed RSSI data, coordinates, and AP columns from saved files.

In [4]:
# Specify building ID
building_id = 1

# Load preprocessed data (uses pickle for fast loading)
rssi_train, coords_train, rssi_val, coords_val, ap_columns = load_preprocessed_data(
    building_id=building_id,
    use_pickle=True  # True = fast (pickle), False = slower (Excel)
)

# Initialize and fit the coordinate scaler
scaler_coords = MinMaxScaler()
scaler_coords.fit(coords_train)

print("\n" + "="*60)
print("PREPROCESSED DATA SUMMARY")
print("="*60)
print(f"Building ID: {building_id}")
print(f"Training samples: {rssi_train.shape[0]}")
print(f"Validation samples: {rssi_val.shape[0]}")
print(f"Number of APs: {len(ap_columns)}")
print(f"\nRSSI Training shape: {rssi_train.shape}")
print(f"Coordinates Training shape: {coords_train.shape}")
print(f"RSSI Validation shape: {rssi_val.shape}")
print(f"Coordinates Validation shape: {coords_val.shape}")
print("="*60)

[OK] Loaded preprocessed data from pickle: C:\Users\Mohamed Khalil\Desktop\Quantum-Optimization-In-AP-Selection\data\output_data\preprocessed_data\preprocessed_building_1.pkl
  Training samples: 5196
  Validation samples: 307
  Number of APs: 520

PREPROCESSED DATA SUMMARY
Building ID: 1
Training samples: 5196
Validation samples: 307
Number of APs: 520

RSSI Training shape: (5196, 520)
Coordinates Training shape: (5196, 3)
RSSI Validation shape: (307, 520)
Coordinates Validation shape: (307, 3)


## Step 2: Load Importance Scores

This loads all pre-computed importance metrics from saved CSV files.

In [5]:
# Load all importance dictionaries at once
importance_dicts, _ = load_all_precomputed_data()

# Access individual importance methods
importance_entropy = importance_dicts['entropy']
importance_average = importance_dicts['average']
importance_median = importance_dicts['median']
importance_max = importance_dicts['max']
importance_variance = importance_dicts['variance']
importance_mutual_info = importance_dicts['mutual_info']

print("\n" + "="*60)
print("IMPORTANCE SCORES SUMMARY")
print("="*60)
print(f"Entropy importance: {len(importance_entropy)} APs")
print(f"Average importance: {len(importance_average)} APs")
print(f"Median importance: {len(importance_median)} APs")
print(f"Max importance: {len(importance_max)} APs")
print(f"Variance importance: {len(importance_variance)} APs")
print(f"Mutual Info importance: {len(importance_mutual_info)} APs")
print("="*60)

# Show top 5 APs for each method
print("\nTop 5 APs by Entropy Importance:")
top_entropy = sorted(importance_entropy.items(), key=lambda x: x[1], reverse=True)[:5]
for ap, score in top_entropy:
    print(f"  {ap}: {score:.4f}")

print("\nTop 5 APs by Mutual Information:")
top_mi = sorted(importance_mutual_info.items(), key=lambda x: x[1], reverse=True)[:5]
for ap, score in top_mi:
    print(f"  {ap}: {score:.4f}")

Loading pre-computed importance scores and redundancy matrix

Loading importance scores...
[OK] Loaded 520 APs for entropy importance
[OK] Loaded 520 APs for average importance
[OK] Loaded 520 APs for median importance
[OK] Loaded 520 APs for max importance
[OK] Loaded 520 APs for variance importance
[OK] Loaded 520 APs for mutual_info importance

Loading redundancy matrix...
[OK] Loaded redundancy matrix with shape: (520, 520)

[OK] All data loaded successfully!

IMPORTANCE SCORES SUMMARY
Entropy importance: 520 APs
Average importance: 520 APs
Median importance: 520 APs
Max importance: 520 APs
Variance importance: 520 APs
Mutual Info importance: 520 APs

Top 5 APs by Entropy Importance:
  WAP248: 1.5819
  WAP107: 1.4834
  WAP108: 1.4807
  WAP167: 1.4132
  WAP166: 1.3899

Top 5 APs by Mutual Information:
  WAP167: 0.3408
  WAP166: 0.3356
  WAP179: 0.3326
  WAP104: 0.3307
  WAP119: 0.3261


## Step 3: Load Redundancy Matrix

This loads the pre-computed redundancy matrix from saved files.

In [6]:
# Load redundancy matrix (second return value from load_all_precomputed_data)
_, redundancy_matrix = load_all_precomputed_data()

print("\n" + "="*60)
print("REDUNDANCY MATRIX SUMMARY")
print("="*60)
print(f"Matrix shape: {redundancy_matrix.shape}")
print(f"Matrix type: {type(redundancy_matrix)}")
print(f"\nRedundancy statistics:")
print(f"  Mean redundancy: {redundancy_matrix.values.mean():.4f}")
print(f"  Min redundancy: {redundancy_matrix.values.min():.4f}")
print(f"  Max redundancy: {redundancy_matrix.values.max():.4f}")
print(f"  Median redundancy: {np.median(redundancy_matrix.values):.4f}")
print("="*60)

# Show a sample of the redundancy matrix
print("\nSample of redundancy matrix (first 5x5):")
print(redundancy_matrix.iloc[:5, :5])

Loading pre-computed importance scores and redundancy matrix

Loading importance scores...
[OK] Loaded 520 APs for entropy importance
[OK] Loaded 520 APs for average importance
[OK] Loaded 520 APs for median importance
[OK] Loaded 520 APs for max importance
[OK] Loaded 520 APs for variance importance
[OK] Loaded 520 APs for mutual_info importance

Loading redundancy matrix...
[OK] Loaded redundancy matrix with shape: (520, 520)

[OK] All data loaded successfully!

REDUNDANCY MATRIX SUMMARY
Matrix shape: (520, 520)
Matrix type: <class 'pandas.core.frame.DataFrame'>

Redundancy statistics:
  Mean redundancy: nan
  Min redundancy: nan
  Max redundancy: nan
  Median redundancy: nan

Sample of redundancy matrix (first 5x5):
        WAP001  WAP002  WAP003  WAP004  WAP005
WAP001     NaN     NaN     NaN     NaN     NaN
WAP002     NaN     NaN     NaN     NaN     NaN
WAP003     NaN     NaN     NaN     NaN     NaN
WAP004     NaN     NaN     NaN     NaN     NaN
WAP005     NaN     NaN     NaN     N

## Step 4: Verification and Summary

Verify that all data has been loaded correctly and is ready for use in the optimization pipeline.

In [7]:
# Verify all components are loaded
print("\n" + "="*60)
print("PIPELINE DATA VERIFICATION")
print("="*60)

checks = [
    ("Preprocessed training data", rssi_train is not None and len(rssi_train) > 0),
    ("Preprocessed validation data", rssi_val is not None and len(rssi_val) > 0),
    ("Training coordinates", coords_train is not None and len(coords_train) > 0),
    ("Validation coordinates", coords_val is not None and len(coords_val) > 0),
    ("AP columns", ap_columns is not None and len(ap_columns) > 0),
    ("Entropy importance", len(importance_entropy) > 0),
    ("Average importance", len(importance_average) > 0),
    ("Median importance", len(importance_median) > 0),
    ("Max importance", len(importance_max) > 0),
    ("Variance importance", len(importance_variance) > 0),
    ("Mutual info importance", len(importance_mutual_info) > 0),
    ("Redundancy matrix", redundancy_matrix is not None and redundancy_matrix.shape[0] > 0),
]

all_passed = True
for check_name, result in checks:
    status = "✓" if result else "✗"
    print(f"{status} {check_name}")
    if not result:
        all_passed = False

print("="*60)
if all_passed:
    print("\n✓ ALL DATA LOADED SUCCESSFULLY!")
    print("\nThe pipeline is ready. You can now:")
    print("  1. Run QUBO optimization with different importance metrics")
    print("  2. Train ML models on selected AP subsets")
    print("  3. Evaluate positioning accuracy")
else:
    print("\n✗ SOME DATA FAILED TO LOAD. Please check the above errors.")
print("="*60)


PIPELINE DATA VERIFICATION
✓ Preprocessed training data
✓ Preprocessed validation data
✓ Training coordinates
✓ Validation coordinates
✓ AP columns
✓ Entropy importance
✓ Average importance
✓ Median importance
✓ Max importance
✓ Variance importance
✓ Mutual info importance
✓ Redundancy matrix

✓ ALL DATA LOADED SUCCESSFULLY!

The pipeline is ready. You can now:
  1. Run QUBO optimization with different importance metrics
  2. Train ML models on selected AP subsets
  3. Evaluate positioning accuracy


## Step 5: Load System Parameters

Load normalization parameters and configure QUBO settings.

In [8]:
# Load system parameters from CSV
system_params_path = Path('../../data') / 'system_input' / 'system_parameters.csv'
system_params_df = pd.read_csv(system_params_path)

# Convert to dictionary for easy access
system_params_dict = dict(zip(system_params_df['Parameter'], system_params_df['Value']))

# Extract parameters
LON_MIN = system_params_dict['LON_MIN']
LON_MAX = system_params_dict['LON_MAX']
LAT_MIN = system_params_dict['LAT_MIN']
LAT_MAX = system_params_dict['LAT_MAX']
FLOOR_HEIGHT = system_params_dict['FLOOR_HEIGHT']

# QUBO parameters
k = 20  # Number of APs to select
alpha = 0.9  # Importance vs redundancy trade-off (higher = more importance weight)
penalty = 2.0  # Penalty for violating the k constraint

print("✓ System parameters loaded from CSV:")
print(f"  LON_MIN: {LON_MIN}")
print(f"  LON_MAX: {LON_MAX}")
print(f"  LAT_MIN: {LAT_MIN}")
print(f"  LAT_MAX: {LAT_MAX}")
print(f"  FLOOR_HEIGHT: {FLOOR_HEIGHT}")
print(f"\nQUBO parameters:")
print(f"  k (num APs to select): {k}")
print(f"  alpha (importance weight): {alpha}")
print(f"  penalty: {penalty}")

✓ System parameters loaded from CSV:
  LON_MIN: -7578.46197155118
  LON_MAX: -7404.491683006287
  LAT_MIN: 4864809.458700001
  LAT_MAX: 4864959.505251184
  FLOOR_HEIGHT: 3.0

QUBO parameters:
  k (num APs to select): 20
  alpha (importance weight): 0.9
  penalty: 2.0


## Step 6: Run QUBO Optimization with Different Importance Metrics

This section runs QUBO optimization using each importance metric and selects the top k APs.

In [9]:
# Dictionary to store results
results = {}

# Importance methods to test
importance_methods = {
    'mutual_info': importance_mutual_info,
    'entropy': importance_entropy,
    'average': importance_average,
    'max': importance_max,
    'variance': importance_variance
}

print("="*60)
print("RUNNING QUBO OPTIMIZATION FOR EACH IMPORTANCE METRIC")
print("="*60)

# Run QUBO for each importance metric
for label, imp_dict in importance_methods.items():
    print(f"\n[{label.upper()}]")
    
    # Check for zero importance scores
    nonzero_scores = [v for v in imp_dict.values() if v > 0]
    if len(nonzero_scores) == 0:
        print(f"  ✗ Skipped: all importance scores are zero or negative.")
        continue

    # 1. Formulate QUBO
    print(f"  → Formulating QUBO (k={k}, alpha={alpha}, penalty={penalty})...")
    Q, relevant_aps, offset = formulate_qubo(imp_dict, redundancy_matrix, k, alpha, penalty)
    
    if len(relevant_aps) == 0:
        print(f"  ✗ Skipped: no relevant APs selected after QUBO formulation.")
        continue

    # 2. Solve QUBO with OpenJij
    print(f"  → Solving QUBO with OpenJij SQA...")
    selected_indices, duration = solve_qubo_with_openjij(Q)
    
    if len(selected_indices) == 0:
        print(f"  ✗ Skipped: QUBO solver did not select any APs.")
        continue

    # Map indices to AP names
    selected_aps = [relevant_aps[i] for i in selected_indices]
    
    # Store preliminary results
    results[label] = {
        'selected_aps': selected_aps,
        'num_aps': len(selected_aps),
        'qubo_duration': duration,
        'Q_matrix': Q,
        'relevant_aps': relevant_aps
    }
    
    print(f"  ✓ Selected {len(selected_aps)} APs in {duration:.2f}s")
    print(f"    APs: {', '.join(selected_aps[:5])}{'...' if len(selected_aps) > 5 else ''}")

print("\n" + "="*60)
print(f"✓ QUBO optimization completed for {len(results)} methods")
print("="*60)

RUNNING QUBO OPTIMIZATION FOR EACH IMPORTANCE METRIC

[MUTUAL_INFO]
  → Formulating QUBO (k=20, alpha=0.9, penalty=2.0)...
Formulating enhanced QUBO for k=20 APs selection...
Done
  → Solving QUBO with OpenJij SQA...

Solving QUBO with OpenJij Simulated Quantum Annealing (SQA)...
OpenJij completed in 31.5951 seconds
  ✓ Selected 20 APs in 31.60s
    APs: WAP015, WAP051, WAP091, WAP102, WAP106...

[ENTROPY]
  → Formulating QUBO (k=20, alpha=0.9, penalty=2.0)...
Formulating enhanced QUBO for k=20 APs selection...
Done
  → Solving QUBO with OpenJij SQA...

Solving QUBO with OpenJij Simulated Quantum Annealing (SQA)...
OpenJij completed in 34.9735 seconds
  ✓ Selected 20 APs in 34.97s
    APs: WAP016, WAP037, WAP090, WAP091, WAP103...

[AVERAGE]
  → Formulating QUBO (k=20, alpha=0.9, penalty=2.0)...
Formulating enhanced QUBO for k=20 APs selection...
Done
  → Solving QUBO with OpenJij SQA...

Solving QUBO with OpenJij Simulated Quantum Annealing (SQA)...
OpenJij completed in 35.7068 second

## Step 7: Train ML Models on Selected AP Subsets

For each importance method, train a Random Forest regressor using the selected APs.

In [10]:
print("="*60)
print("TRAINING ML MODELS")
print("="*60)

# Train models for each method
for label in results.keys():
    print(f"\n[{label.upper()}]")
    
    selected_aps = results[label]['selected_aps']
    
    # Train Random Forest regressor
    print(f"  → Training Random Forest with {len(selected_aps)} APs...")
    models, predictions = train_regressor(
        rssi_train, coords_train, 
        rssi_val, coords_val, 
        selected_aps
    )
    
    # Get validation predictions
    preds = predictions['rf_val']
    
    # Store model and predictions
    results[label]['models'] = models
    results[label]['predictions'] = predictions
    results[label]['preds_val'] = preds
    
    print(f"  ✓ Model trained successfully")
    if 'rf' in models:
        oob_score = models['rf'].estimators_[0].oob_score_ if hasattr(models['rf'].estimators_[0], 'oob_score_') else 'N/A'
        print(f"    Validation predictions shape: {preds.shape}")

print("\n" + "="*60)
print(f"✓ ML models trained for {len(results)} methods")
print("="*60)

TRAINING ML MODELS

[MUTUAL_INFO]
  → Training Random Forest with 20 APs...
Training random forest regressor...
✓ Enhanced Random Forest trained
   Average OOB Score: 0.9054
  ✓ Model trained successfully

[ENTROPY]
  → Training Random Forest with 20 APs...
Training random forest regressor...
✓ Enhanced Random Forest trained
   Average OOB Score: 0.9403
  ✓ Model trained successfully

[AVERAGE]
  → Training Random Forest with 20 APs...
Training random forest regressor...
✓ Enhanced Random Forest trained
   Average OOB Score: 0.8933
  ✓ Model trained successfully

[MAX]
  → Training Random Forest with 20 APs...
Training random forest regressor...
✓ Enhanced Random Forest trained
   Average OOB Score: 0.8725
  ✓ Model trained successfully

[VARIANCE]
  → Training Random Forest with 20 APs...
Training random forest regressor...
✓ Enhanced Random Forest trained
   Average OOB Score: 0.8128
  ✓ Model trained successfully

✓ ML models trained for 5 methods


## Step 8: Evaluate Positioning Accuracy

Calculate comprehensive metrics for each method including 3D positioning error and floor accuracy.

In [11]:
print("="*60)
print("EVALUATING POSITIONING ACCURACY")
print("="*60)

# Evaluate each method
for label in results.keys():
    print(f"\n[{label.upper()}]")
    
    preds = results[label]['preds_val']
    
    # Calculate comprehensive metrics
    print(f"  → Calculating positioning metrics...")
    _, _, metrics = calculate_comprehensive_metrics(
        coords_val, preds,
        LON_MIN, LON_MAX,
        LAT_MIN, LAT_MAX,
        FLOOR_HEIGHT
    )
    
    # Store metrics - now including all 3 floor accuracy levels
    results[label]['mean_3d_error'] = metrics['real_mean_m']
    results[label]['median_3d_error'] = metrics['real_median_m']
    results[label]['min_error'] = metrics['real_min_m']
    results[label]['max_error'] = metrics['real_max_m']
    results[label]['floor_accuracy_0'] = metrics['floor_accuracy_0']
    results[label]['floor_accuracy_1'] = metrics['floor_accuracy_1']
    results[label]['floor_accuracy_2'] = metrics['floor_accuracy_2']
    results[label]['all_metrics'] = metrics
    
    print(f"  ✓ Metrics calculated:")
    print(f"    Mean 3D Error: {metrics['real_mean_m']:.2f} m")
    print(f"    Median 3D Error: {metrics['real_median_m']:.2f} m")
    print(f"    Floor Accuracy (exact): {metrics['floor_accuracy_0']:.2%}")
    print(f"    Floor Accuracy (±1): {metrics['floor_accuracy_1']:.2%}")
    print(f"    Floor Accuracy (±2): {metrics['floor_accuracy_2']:.2%}")

print("\n" + "="*60)
print(f"✓ Evaluation completed for {len(results)} methods")
print("="*60)

EVALUATING POSITIONING ACCURACY

[MUTUAL_INFO]
  → Calculating positioning metrics...
  ✓ Metrics calculated:
    Mean 3D Error: 18.58 m
    Median 3D Error: 13.62 m
    Floor Accuracy (exact): 69.38%
    Floor Accuracy (±1): 97.39%
    Floor Accuracy (±2): 100.00%

[ENTROPY]
  → Calculating positioning metrics...
  ✓ Metrics calculated:
    Mean 3D Error: 17.00 m
    Median 3D Error: 11.58 m
    Floor Accuracy (exact): 66.12%
    Floor Accuracy (±1): 98.37%
    Floor Accuracy (±2): 100.00%

[AVERAGE]
  → Calculating positioning metrics...
  ✓ Metrics calculated:
    Mean 3D Error: 18.29 m
    Median 3D Error: 12.36 m
    Floor Accuracy (exact): 58.63%
    Floor Accuracy (±1): 96.09%
    Floor Accuracy (±2): 99.67%

[MAX]
  → Calculating positioning metrics...
  ✓ Metrics calculated:
    Mean 3D Error: 17.93 m
    Median 3D Error: 13.98 m
    Floor Accuracy (exact): 65.80%
    Floor Accuracy (±1): 96.09%
    Floor Accuracy (±2): 99.67%

[VARIANCE]
  → Calculating positioning metrics...

## Step 9: Compare Results Across All Methods

Create a comprehensive comparison table and visualize the results.

In [12]:
# Create results DataFrame
results_data = []
for label, data in results.items():
    # Abbreviate Selected_APs list for display (first 5 APs)
    selected_aps_full = data['selected_aps']
    if len(selected_aps_full) > 5:
        selected_aps_display = ', '.join(selected_aps_full[:5]) + ', ...'
    else:
        selected_aps_display = ', '.join(selected_aps_full)
    
    results_data.append({
        'Importance_Method': label.upper(),
        'Num_APs': data['num_aps'],
        'Selected_APs': ', '.join(selected_aps_full),  # Full list for Excel
        'Selected_APs_Display': selected_aps_display,  # Abbreviated for display
        'Mean_3D_Error_m': data['mean_3d_error'],
        'Median_3D_Error_m': data['median_3d_error'],
        'Min_Error_m': data['min_error'],
        'Max_Error_m': data['max_error'],
        'Floor_Accuracy_0': data['floor_accuracy_0'],
        'Floor_Accuracy_1': data['floor_accuracy_1'],
        'Floor_Accuracy_2': data['floor_accuracy_2']
    })

results_df = pd.DataFrame(results_data)

# Sort by mean 3D error (best first)
results_df = results_df.sort_values('Mean_3D_Error_m', ascending=True)

print("="*100)
print("RESULTS COMPARISON - ALL IMPORTANCE METHODS")
print("="*100)
print("\nSummary Table (sorted by Mean 3D Error):")
# Display abbreviated version
display_cols = ['Importance_Method', 'Num_APs', 'Selected_APs_Display', 
                'Mean_3D_Error_m', 'Median_3D_Error_m', 'Min_Error_m', 'Max_Error_m',
                'Floor_Accuracy_0', 'Floor_Accuracy_1', 'Floor_Accuracy_2']
print(results_df[display_cols].to_string(index=False))

print("\n" + "="*100)
print("BEST PERFORMING METHOD")
print("="*100)
best_method = results_df.iloc[0]
print(f"Method: {best_method['Importance_Method']}")
print(f"Mean 3D Error: {best_method['Mean_3D_Error_m']:.2f} m")
print(f"Median 3D Error: {best_method['Median_3D_Error_m']:.2f} m")
print(f"Floor Accuracy (exact): {best_method['Floor_Accuracy_0']:.2%}")
print(f"Floor Accuracy (±1 floor): {best_method['Floor_Accuracy_1']:.2%}")
print(f"Floor Accuracy (±2 floors): {best_method['Floor_Accuracy_2']:.2%}")
print(f"Number of APs: {best_method['Num_APs']}")
print("="*100)

RESULTS COMPARISON - ALL IMPORTANCE METHODS

Summary Table (sorted by Mean 3D Error):
Importance_Method  Num_APs                        Selected_APs_Display  Mean_3D_Error_m  Median_3D_Error_m  Min_Error_m  Max_Error_m  Floor_Accuracy_0  Floor_Accuracy_1  Floor_Accuracy_2
          ENTROPY       20 WAP016, WAP037, WAP090, WAP091, WAP103, ...        16.997920          11.578227     0.206596    75.044313          0.661238          0.983713          1.000000
         VARIANCE       20 WAP089, WAP090, WAP091, WAP101, WAP108, ...        17.587196          13.526260     0.951917    70.356851          0.664495          0.944625          0.996743
              MAX       20 WAP046, WAP084, WAP090, WAP102, WAP105, ...        17.933808          13.978657     0.496442    72.255256          0.657980          0.960912          0.996743
          AVERAGE       20 WAP059, WAP091, WAP101, WAP103, WAP105, ...        18.288905          12.355929     0.695355   124.481938          0.586319          0.9609

## Step 10: Save Results

Save the results to Excel and CSV files for further analysis.

In [13]:
# Create output directory
output_dir = Path('../../data') / 'results'
output_dir.mkdir(parents=True, exist_ok=True)

# Select columns to save (exclude the display-only column)
save_cols = ['Importance_Method', 'Num_APs', 'Selected_APs', 
             'Mean_3D_Error_m', 'Median_3D_Error_m', 'Min_Error_m', 'Max_Error_m',
             'Floor_Accuracy_0', 'Floor_Accuracy_1', 'Floor_Accuracy_2']

results_df_save = results_df[save_cols]

# Save as Excel
excel_path = output_dir / 'pipeline_experiment_results.xlsx'
results_df_save.to_excel(excel_path, index=False)
print(f"✓ Results saved to Excel: {excel_path}")

# Save as CSV
csv_path = output_dir / 'pipeline_experiment_results.csv'
results_df_save.to_csv(csv_path, index=False)
print(f"✓ Results saved to CSV: {csv_path}")

# Also save a detailed version with selected APs
detailed_path = output_dir / 'pipeline_experiment_detailed.xlsx'
with pd.ExcelWriter(detailed_path, engine='openpyxl') as writer:
    # Summary sheet with all metrics
    results_df_save.to_excel(writer, sheet_name='Summary', index=False)
    
    # Individual sheets for each method's selected APs
    for label, data in results.items():
        ap_df = pd.DataFrame({
            'AP_Name': data['selected_aps'],
            'Index': range(len(data['selected_aps']))
        })
        sheet_name = f"{label.upper()}_APs"[:31]  # Excel sheet name limit
        ap_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"✓ Detailed results saved to: {detailed_path}")

print("\n" + "="*60)
print("✓ ALL RESULTS SAVED SUCCESSFULLY!")
print("="*60)
print("\nSaved columns:")
for col in save_cols:
    print(f"  - {col}")
print("\nNote: QUBO_Duration_s has been removed from output as requested.")

✓ Results saved to Excel: ..\..\data\results\pipeline_experiment_results.xlsx
✓ Results saved to CSV: ..\..\data\results\pipeline_experiment_results.csv
✓ Detailed results saved to: ..\..\data\results\pipeline_experiment_detailed.xlsx

✓ ALL RESULTS SAVED SUCCESSFULLY!

Saved columns:
  - Importance_Method
  - Num_APs
  - Selected_APs
  - Mean_3D_Error_m
  - Median_3D_Error_m
  - Min_Error_m
  - Max_Error_m
  - Floor_Accuracy_0
  - Floor_Accuracy_1
  - Floor_Accuracy_2

Note: QUBO_Duration_s has been removed from output as requested.
